# Visual Inspection and Planning

The purpose of this section is to get an idea of what the data looks like, what it should look like and how to get it from what it looks like to what it should look like.

In [1]:
# Import necessary modules
import pandas as pd
import json
import re

### Load DataFrame

In [2]:
# Load data onto a pandas DataFrame object

rawfifa = pd.read_csv(r"C:\Users\T. Barnes\Documents\Data Science\DataCleaningChallenge\Working_on\fifa21 raw data v2.csv")
rawfifa.head()

C:\Users\T. Barnes\AppData\Local\Temp\ipykernel_5080\3582111910.py:3: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  rawfifa = pd.read_csv(r"C:\Users\T. Barnes\Documents\Data Science\DataCleaningChallenge\Working_on\fifa21 raw data v2.csv")


,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,158023,L. Messi,Lionel Messi,https://cdn.sofifa.com/players/158/023/21_60.png,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,33,93,93,\n\n\n\nFC Barcelona,...,Medium,Low,5 ★,85,92,91,95,38,65,771
1,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,https://cdn.sofifa.com/players/020/801/21_60.png,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,35,92,92,\n\n\n\nJuventus,...,High,Low,5 ★,89,93,81,89,35,77,562
2,200389,J. Oblak,Jan Oblak,https://cdn.sofifa.com/players/200/389/21_60.png,http://sofifa.com/player/200389/jan-oblak/210006/,Slovenia,27,91,93,\n\n\n\nAtlético Madrid,...,Medium,Medium,3 ★,87,92,78,90,52,90,150
3,192985,K. De Bruyne,Kevin De Bruyne,https://cdn.sofifa.com/players/192/985/21_60.png,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,29,91,91,\n\n\n\nManchester City,...,High,High,4 ★,76,86,93,88,64,78,207
4,190871,Neymar Jr,Neymar da Silva Santos Jr.,https://cdn.sofifa.com/players/190/871/21_60.png,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,28,91,91,\n\n\n\nParis Saint-Germain,...,High,Medium,5 ★,91,85,86,94,36,59,595


In [3]:
rawfifa.columns[76]

'Hits'

Note pandas' warning about mixed types in column 76 which is the "Hits" column.

### Create data dictionary

This dictionary is going to be created from the documentation provided alongside the dataset. This documentation was provided as a series of messages on telegram. I have decided to convert them to a more convenient format.

I will copy the documentation into a str variable, and then use that to create a dictionary. This dictionary will be more convenient to reference.

#### Copy documentation into string

In [4]:
# doc to string variable

strdoc = """photoUrl: The URL of the player's photo.
LongName: The full name of the player.
playerUrl: The URL of the player's page on sofifa.com.
Nationality: The nationality of the player.
Positions: The positions the player can play.
Name: The short name of the player.
Age: The age of the player.
OVA: The overall rating of the player in FIFA 21.
POT: The potential rating of the player in FIFA 21.
Team & Contract: The team the player is playing for in FIFA 21, along with their contract details.
ID: The unique identifier for the player.
Height: The height of the player in feet and inches.
Weight: The weight of the player in pounds.
Preferred foot: The preferred foot of the player.
BOV: The best overall rating the player has achieved in their career.
Best Position: The best position the player has played in their career.
Growth: The difference between the potential rating and overall rating of the player.
Joined: The date the player joined their current team in FIFA 21.
Loan Date End: The date the player's loan contract ends.
Value: The market value of the player in FIFA 21.
Wage: The weekly wage of the player in FIFA 21.
Release Clause: The release clause value of the player in FIFA 21.
Attacking: The attacking attributes of the player.
Crossing: The crossing attribute of the player.
Finishing: The finishing attribute of the player.
Heading Accuracy: The heading accuracy attribute of the player.
Short Passing: The short passing attribute of the player.
Volleys: The volleys attribute of the player.
Skill: The skill attributes of the player.
Dribbling: The dribbling attribute of the player.
Curve: The curve attribute of the player.
FK Accuracy: The free kick accuracy attribute of the player.
Long Passing: The long passing attribute of the player.
Ball Control: The ball control attribute of the player.
Movement: The movement attributes of the player.
Acceleration: The acceleration attribute of the player.
Sprint Speed: The sprint speed attribute of the player.
Agility: The agility attribute of the player.
Reactions: The reactions attribute of the player.
Balance: The balance attribute of the player.
Power: The power attributes of the player.
Shot Power: The shot power attribute of the player.
Jumping: The jumping attribute of the player.
Stamina: The stamina attribute of the player.
Strength: The strength attribute of the player.
Long Shots: The long shots attribute of the player.
Mentality: The mentality attributes of the player.
Aggression: The aggression attribute of the player.
Interceptions: The interceptions attribute of the player.
Positioning: The positioning attribute of the player.
Vision: The vision attribute of the player.
Penalties: The penalties attribute of the player.
Composure: The composure attribute of the player.
Defending: The defending attributes of the player.
Marking: The marking attribute of the player.
Standing Tackle: The standing tackle attribute of the player.
Sliding Tackle: The sliding tackle attribute of the player.
Goalkeeping: The goalkeeping attributes of the player.
GK Diving: The goalkeeper diving attribute of the player.
GK Handling: The goalkeeper handling attribute of the player.
GK Kicking: The goalkeeper kicking attribute of the player.
GK Positioning: The goalkeeper positioning attribute of the player.
GK Reflexes: This refers to the goalkeeper's ability to react and make saves quickly.
Total Stats: This refers to the overall rating of the player based on their performance in all areas of the game.
Base Stats: This refers to the player's rating in the six main areas of the game: Pace, Shooting, Passing, Dribbling, Defending, and Physicality.
W/F: This refers to the player's weaker foot ability.
SM: This refers to the player's skill moves ability.
A/W: This refers to the player's attacking work rate. It measures how frequently the player participates in attacking actions, such as making runs or positioning themselves in the opponent's half.
D/W: This refers to the player's defensive work rate. It measures how frequently the player participates in defensive actions, such as tracking back or making tackles.
IR: This refers to the player's injury resistance. It measures the player's ability to avoid injuries and how quickly they recover from them.
PAC: This refers to the player's pace or speed attribute. It measures how quickly the player can move with and without the ball.
SHO: This refers to the player's shooting ability. It measures the player's accuracy and power when shooting the ball.
PAS: This refers to the player's passing ability. It measures the player's accuracy and range when passing the ball.
DRI: This refers to the player's dribbling ability. It measures the player's agility, balance, and ball control when dribbling the ball.
DEF: This refers to the player's defensive ability. It measures the player's ability to tackle, intercept, and defend against opposing players.
PHY: This refers to the player's physicality or strength. It measures the player's ability to win physical battles and maintain possession of the ball.
Hits: This refers to the number of times the player's profile has been viewed on the website."""



#### Create dictionary

In [5]:
# Split string into list of key-value pairs

keyvalues = strdoc.split(".\n")

# Use dict comprehension to loop through list for dictionary

docdict = {re.sub("[^a-zA-Z]", "_", x.split(": ")[0]).replace("___", "_").lower(): x.split(": ")[1] for x in keyvalues}
len(docdict)

77

The sub() function is used here to get rid of uppercase letters and characters not in the alphabet.

This is a style choice.

In the end, my dict keys look like so...

In [6]:
docdict.keys()

dict_keys(['photourl', 'longname', 'playerurl', 'nationality', 'positions', 'name', 'age', 'ova', 'pot', 'team_contract', 'id', 'height', 'weight', 'preferred_foot', 'bov', 'best_position', 'growth', 'joined', 'loan_date_end', 'value', 'wage', 'release_clause', 'attacking', 'crossing', 'finishing', 'heading_accuracy', 'short_passing', 'volleys', 'skill', 'dribbling', 'curve', 'fk_accuracy', 'long_passing', 'ball_control', 'movement', 'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance', 'power', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots', 'mentality', 'aggression', 'interceptions', 'positioning', 'vision', 'penalties', 'composure', 'defending', 'marking', 'standing_tackle', 'sliding_tackle', 'goalkeeping', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes', 'total_stats', 'base_stats', 'w_f', 'sm', 'a_w', 'd_w', 'ir', 'pac', 'sho', 'pas', 'dri', 'def', 'phy', 'hits'])

While I'm at it, I'll give my column headers the same treatment, in the interest of fairness.

In [7]:
rawfifa.columns = [re.sub("[^a-zA-Z]", "_", x).replace("___", "_").strip().strip("_").lower() for x in rawfifa.columns]
rawfifa.columns

Index(['id', 'name', 'longname', 'photourl', 'playerurl', 'nationality', 'age',
       'ova', 'pot', 'club', 'contract', 'positions', 'height', 'weight',
       'preferred_foot', 'bov', 'best_position', 'joined', 'loan_date_end',
       'value', 'wage', 'release_clause', 'attacking', 'crossing', 'finishing',
       'heading_accuracy', 'short_passing', 'volleys', 'skill', 'dribbling',
       'curve', 'fk_accuracy', 'long_passing', 'ball_control', 'movement',
       'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance',
       'power', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'mentality', 'aggression', 'interceptions', 'positioning', 'vision',
       'penalties', 'composure', 'defending', 'marking', 'standing_tackle',
       'sliding_tackle', 'goalkeeping', 'gk_diving', 'gk_handling',
       'gk_kicking', 'gk_positioning', 'gk_reflexes', 'total_stats',
       'base_stats', 'w_f', 'sm', 'a_w', 'd_w', 'ir', 'pac', 'sho', 'pas',
       'dri', 'def', '

#### Check for uniformity in dictionary and dataframe

column names and dictionary keys need to be the same. Otherwise, referencing the documentation will become problematic

In [8]:
# Check that column names are the same as dict keys
# the indf and indict lists will be empty if they are the same

indf = []
indict = []
for x in rawfifa.columns:
    if x not in docdict.keys():
        indf.append(x)
        
for x in docdict.keys():
    if x not in rawfifa.columns:
        indict.append(x)

In [9]:
# Check each list

print("Outlier keys in dict\n")

for x in indict:
    print(f"{x}\n")

print("Outlier columns in dataframe\n")
for x in indf:
    print(f"{x}\n")

Outlier keys in dict

team_contract

growth

Outlier columns in dataframe

club

contract



They are not uniform. However, after a little investigation, I've discovered that the "team_contract" column has been split into the "club" and "contract" columns in the current dataset, while the "growth" column has been dropped.

So I have decided to leave them as is, because I believe the disparity will not affect my work negatively.

### Function to look up column documentation

The column documentation is essentially just a description of what the column is, or rather, what it should be. So I'm going to be thinking of them as their respective column's laws.

Why do I need to point this out? Because, the reasoning might be a bit tacked on, but it is also a good excuse for me to name my function "coleslaw"

In [10]:
def coleslaw(col):
    
    if col in docdict.keys():
        print(docdict[col])
    
    else:
        print("(in Tchalla's voice): We don't have that here")

Stow that away for later and move to a preliminary inspection of the dataset.

In [11]:
# Inspect column info

rawfifa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18979 entries, 0 to 18978
Data columns (total 77 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                18979 non-null  int64 
 1   name              18979 non-null  object
 2   longname          18979 non-null  object
 3   photourl          18979 non-null  object
 4   playerurl         18979 non-null  object
 5   nationality       18979 non-null  object
 6   age               18979 non-null  int64 
 7   ova               18979 non-null  int64 
 8   pot               18979 non-null  int64 
 9   club              18979 non-null  object
 10  contract          18979 non-null  object
 11  positions         18979 non-null  object
 12  height            18979 non-null  object
 13  weight            18979 non-null  object
 14  preferred_foot    18979 non-null  object
 15  bov               18979 non-null  int64 
 16  best_position     18979 non-null  object
 17  joined      

The above tells me that only two columns have null values:

* "loan_date_end" column and
* "hits" column

So next, let me find out what these columns are about.

In [12]:
# loan_date_end column

coleslaw("loan_date_end")

The date the player's loan contract ends


From the description above, I can deduce that a null value in this column means that a player's contract is not a loan.

I think leaving it as null is appropriate in this case.

One thing to check might be if the year in this date corresponds to the year in the "contract" column

In [13]:
# hits column

coleslaw("hits")

This refers to the number of times the player's profile has been viewed on the website.


Given the above, a null might mean that a player's page has not been visited, or that a player does not have a page.

One way to narrow down these possibilities is to check if there is any player without a player page

In [14]:
# Column with player's page data
coleslaw("playerurl")

The URL of the player's page on sofifa.com


In [15]:
# Taking a look at the column

rawfifa.head()['playerurl']

0    http://sofifa.com/player/158023/lionel-messi/2...
1    http://sofifa.com/player/20801/c-ronaldo-dos-s...
2    http://sofifa.com/player/200389/jan-oblak/210006/
3    http://sofifa.com/player/192985/kevin-de-bruyn...
4    http://sofifa.com/player/190871/neymar-da-silv...
Name: playerurl, dtype: object

All urls start with "http://sofifa.com/", so I think it is safe to assume that any row not following this pattern will not lead to a player's page.

In [16]:
# Check "playerurl" column for rows without url pattern

rawfifa[~rawfifa['playerurl'].str.contains("http://sofifa.com/")]['playerurl']

Series([], Name: playerurl, dtype: object)

The result is an empty series, so I can assume that all players have pages. Therefore, null values in the "hits" column means that a player has not had a hit on their page.

Therefore, I will be replacing null values in the "hits" column with 0.


### Check for consistent datatypes in hits column

This is just to satisfy myself that whatever pandas was warning me about previously isn't going to be debilitating

In [17]:
# Create a set of all datatypes in "hits" column
dtypes = {type(x) for x in rawfifa.hits}
dtypes

{float, str}

The "hits column contains data of type float and str

In [18]:
# Inspect float values

float_rows = {x for x in rawfifa.hits if type(x) == float}
float_rows

{1.0,
 2.0,
 3.0,
 4.0,
 6.0,
 7.0,
 9.0,
 10.0,
 11.0,
 17.0,
 31.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,


The float values are mostly Null values and so will mostly be dealt with when I substiture Null for 0. The rest of them will be easily converted to ints as there is no need for decimal values here.

In [19]:
str_rows = {x for x in rawfifa.hits if type(x) == str}
str_rows

{'1',
 '1.1K',
 '1.2K',
 '1.3K',
 '1.4K',
 '1.5K',
 '1.6K',
 '1.7K',
 '1.8K',
 '1.9K',
 '10',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '11',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '12',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '13',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138',
 '139',
 '14',
 '140',
 '141',
 '142',
 '143',
 '144',
 '145',
 '146',
 '147',
 '148',
 '149',
 '15',
 '150',
 '151',
 '152',
 '153',
 '154',
 '155',
 '156',
 '157',
 '158',
 '159',
 '16',
 '160',
 '161',
 '162',
 '164',
 '165',
 '166',
 '167',
 '168',
 '169',
 '17',
 '170',
 '171',
 '172',
 '173',
 '174',
 '175',
 '176',
 '177',
 '178',
 '179',
 '18',
 '180',
 '181',
 '182',
 '183',
 '184',
 '186',
 '187',
 '188',
 '189',
 '19',
 '190',
 '191',
 '192',
 '193',
 '194',
 '195',
 '196',
 '197',
 '198',
 '199',
 '1K',
 '2',
 '2.3K',
 '2.5K',
 '2.8K',
 '20',
 '200',
 '20

The string values are standard except for those above 1000 which are represented as fractions of 1000 with a K appended. Also needs to be converted to ints




# Grouping columns

This will involve categorizing columns generally into:

* Dump
* Numericals
* Dates
* Primary Key_string
* Normal string

I will be using their descriptions, names and sometimes value types to do this. Other categories might be added to these as I deem appropriate.

Both the categories above, and the guide that will be developed with them, are going to be used as a loose guide in the cleaning process. They may not be strictly adhered to, depending on the discoveries made during the cleaning process itself.

Also, the following 'Documentation', 'View', 'Value types' and 'Duplicates' cells will be used repeatedly in categorizing the columns, so as not to repeat code. I have left in the results for the 'a_w' column. Extra columns will be added as needed after the 'Changes by Columns' cell.

In [20]:
list(rawfifa.columns)

['id',
 'name',
 'longname',
 'photourl',
 'playerurl',
 'nationality',
 'age',
 'ova',
 'pot',
 'club',
 'contract',
 'positions',
 'height',
 'weight',
 'preferred_foot',
 'bov',
 'best_position',
 'joined',
 'loan_date_end',
 'value',
 'wage',
 'release_clause',
 'attacking',
 'crossing',
 'finishing',
 'heading_accuracy',
 'short_passing',
 'volleys',
 'skill',
 'dribbling',
 'curve',
 'fk_accuracy',
 'long_passing',
 'ball_control',
 'movement',
 'acceleration',
 'sprint_speed',
 'agility',
 'reactions',
 'balance',
 'power',
 'shot_power',
 'jumping',
 'stamina',
 'strength',
 'long_shots',
 'mentality',
 'aggression',
 'interceptions',
 'positioning',
 'vision',
 'penalties',
 'composure',
 'defending',
 'marking',
 'standing_tackle',
 'sliding_tackle',
 'goalkeeping',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_positioning',
 'gk_reflexes',
 'total_stats',
 'base_stats',
 'w_f',
 'sm',
 'a_w',
 'd_w',
 'ir',
 'pac',
 'sho',
 'pas',
 'dri',
 'def',
 'phy',
 'hits']

In [21]:
# Documentation
coleslaw("a_w")

This refers to the player's attacking work rate. It measures how frequently the player participates in attacking actions, such as making runs or positioning themselves in the opponent's half


In [22]:
# View

rawfifa.head(10)['a_w']

0    Medium
1      High
2    Medium
3      High
4      High
5      High
6      High
7    Medium
8      High
9    Medium
Name: a_w, dtype: object

In [23]:
# Value types

all_types = {type(x) for x in rawfifa.a_w}
print(len(all_types))
all_types

1


{str}

In [24]:
# Duplicates
rawfifa[rawfifa.duplicated(subset='a_w')]

,id,name,longname,photourl,playerurl,nationality,age,ova,pot,club,...,a_w,d_w,ir,pac,sho,pas,dri,def,phy,hits
2,200389,J. Oblak,Jan Oblak,https://cdn.sofifa.com/players/200/389/21_60.png,http://sofifa.com/player/200389/jan-oblak/210006/,Slovenia,27,91,93,\n\n\n\nAtlético Madrid,...,Medium,Medium,3 ★,87,92,78,90,52,90,150
3,192985,K. De Bruyne,Kevin De Bruyne,https://cdn.sofifa.com/players/192/985/21_60.png,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,29,91,91,\n\n\n\nManchester City,...,High,High,4 ★,76,86,93,88,64,78,207
4,190871,Neymar Jr,Neymar da Silva Santos Jr.,https://cdn.sofifa.com/players/190/871/21_60.png,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,28,91,91,\n\n\n\nParis Saint-Germain,...,High,Medium,5 ★,91,85,86,94,36,59,595
5,188545,R. Lewandowski,Robert Lewandowski,https://cdn.sofifa.com/players/188/545/21_60.png,http://sofifa.com/player/188545/robert-lewando...,Poland,31,91,91,\n\n\n\nFC Bayern München,...,High,Medium,4 ★,78,91,78,85,43,82,248
6,209331,M. Salah,Mohamed Salah,https://cdn.sofifa.com/players/209/331/21_60.png,http://sofifa.com/player/209331/mohamed-salah/...,Egypt,28,90,90,\n\n\n\nLiverpool,...,High,Medium,3 ★,93,86,81,90,45,75,246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18974,247223,Xia Ao,Ao Xia,https://cdn.sofifa.com/players/247/223/21_60.png,http://sofifa.com/player/247223/ao-xia/210006/,China PR,21,47,55,\n\n\n\nWuhan Zall,...,Medium,Medium,1 ★,64,28,26,38,48,51,NaN
18975,258760,B. Hough,Ben Hough,https://cdn.sofifa.com/players/258/760/21_60.png,http://sofifa.com/player/258760/ben-hough/210006/,England,17,47,67,\n\n\n\nOldham Athletic,...,Medium,Medium,1 ★,64,40,48,49,35,45,NaN
18976,252757,R. McKinley,Ronan McKinley,https://cdn.sofifa.com/players/252/757/21_60.png,http://sofifa.com/player/252757/ronan-mckinley...,England,18,47,65,\n\n\n\nDerry City,...,Medium,Medium,1 ★,63,39,44,46,40,53,NaN
18977,243790,Wang Zhen'ao,Zhen'ao Wang,https://cdn.sofifa.com/players/243/790/21_60.png,http://sofifa.com/player/243790/zhenao-wang/21...,China PR,20,47,57,\n\n\n\nDalian YiFang FC,...,Medium,Medium,1 ★,58,49,41,49,30,44,NaN


# Changes by columns
This is the final document for this preliminary stage

### id - pk_string
* change type to string

### name and longname - category_string
Neither are unique
* leave as is

### photourl and playerurl - drop
Columns have no impact on any kind of analysis
* drop columns
* maybe keep photourl for visualizations

### nationality - category_string
* Ensure uniformity of values throughout column
* Ensure capitalization

### age, ova, pot, bov - int
* check for impossible values

### club - string
* Uniformity of values
* deal with newline character or any other weird chars

### contract - date
* break into two date columns: "start" and "end"

### positions - string
* Either leave as is or
* Create a player-position table for one-many relationship

### height, weight - int
Looks to be in uniform units
* Remove units
* Convert to ints

### preferred_foot, best_position - category_string
* Ensure uniformity of distinct values

### joined - date
* Check to see if "joined" makes the contract column redundant
* Check for uniform date format

### loan_date_end - date
* Leave null values as is
* Ensure uniform date format

### value, wage, release_clause - int
* Remove symbols and letters
* Convert to int and multiply by appropriate value

### attacking, defending, goalkeeping - int
Has values higher than expected range
* Check if value is sum of related cols

### rawfifa.columns[23:64] - int
* check for all int

### w_f, sm, ir - int
* remove symbol and convert to int

### a_w, d_w - category_string
* uniformity of distinct values

### pac, sho, pas, dri, def, phy - int

### hits - int
* Fillna with 0
* convert floats to int
* convert string to int

In [25]:
rawfifa.iloc[:,23:64].describe()

,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,...,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats
count,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,...,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000
mean,49.688392,45.842405,51.942726,58.768112,42.696507,256.479214,55.587491,47.246957,42.391011,52.725381,...,46.559197,47.705622,45.644449,81.436693,16.416302,16.207124,16.076453,16.217187,16.519627,1595.286949
std,18.131153,19.567081,17.294409,14.519106,17.646937,78.650601,18.761314,18.207790,17.227947,15.178151,...,20.139324,21.367690,20.922087,84.665579,17.554194,16.816305,16.491103,17.002239,17.854079,269.874789
min,6.000000,3.000000,5.000000,7.000000,3.000000,40.000000,5.000000,4.000000,5.000000,5.000000,...,3.000000,5.000000,4.000000,10.000000,2.000000,2.000000,2.000000,2.000000,2.000000,747.000000
25%,38.000000,30.000000,44.000000,54.000000,30.000000,222.000000,49.000000,35.000000,31.000000,43.000000,...,29.000000,27.000000,25.000000,48.000000,8.000000,8.000000,8.000000,8.000000,8.000000,1452.000000
50%,54.000000,49.000000,55.000000,62.000000,44.000000,269.000000,61.000000,49.000000,41.000000,56.000000,...,52.000000,55.000000,52.000000,53.000000,11.000000,11.000000,11.000000,11.000000,11.000000,1627.000000
75%,63.000000,62.000000,64.000000,68.000000,56.000000,310.000000,68.000000,61.000000,55.000000,64.000000,...,63.000000,65.000000,63.000000,59.000000,14.000000,14.000000,14.000000,14.000000,14.000000,1781.000000
max,94.000000,95.000000,93.000000,94.000000,90.000000,470.000000,96.000000,94.000000,94.000000,93.000000,...,94.000000,93.000000,90.000000,440.000000,90.000000,92.000000,93.000000,91.000000,90.000000,2316.000000


In [26]:
rawfifa.iloc[:, 65:76]

,w_f,sm,a_w,d_w,ir,pac,sho,pas,dri,def,phy
0,4 ★,4★,Medium,Low,5 ★,85,92,91,95,38,65
1,4 ★,5★,High,Low,5 ★,89,93,81,89,35,77
2,3 ★,1★,Medium,Medium,3 ★,87,92,78,90,52,90
3,5 ★,4★,High,High,4 ★,76,86,93,88,64,78
4,5 ★,5★,High,Medium,5 ★,91,85,86,94,36,59
...,...,...,...,...,...,...,...,...,...,...,...
18974,2 ★,2★,Medium,Medium,1 ★,64,28,26,38,48,51
18975,2 ★,2★,Medium,Medium,1 ★,64,40,48,49,35,45
18976,2 ★,2★,Medium,Medium,1 ★,63,39,44,46,40,53
18977,3 ★,2★,Medium,Medium,1 ★,58,49,41,49,30,44
